In [29]:
import pandas as pd
from IPython.display import display
def samples_seen_from_str(samples):
    if samples.endswith("M"):
        samples = float(samples[:-1]) * 10**6
    elif samples.endswith("B"):
        samples = float(samples[:-1]) * 10**9
    elif samples.endswith("K"):
        samples = float(samples[:-1]) * 10**3
    return samples
import os
def status(x):
    dirname = os.path.dirname(x.path)
    cur = x.epoch
    last = x.total_epochs
    status = 0

    if os.path.exists(os.path.join(dirname, f"epoch_{last}.pt")):
        status = 1    
    path = x.path.replace(f"epoch_{cur}", f"epoch_{last}")
    if os.path.exists(path):
        status = 2
    return status

#d = d.style.map(highlight)
"""
print
      Gray: not trained
      Yellow: Training not finished
      Blue: Finished training, but not evaluation
      Green: Finished evaluation
"""
#d["status"] = d.apply(lambda di:status(di), axis=1)
def highlight_counts(val):
    if val == 0:
        color = "red"
    else:
        color = "white"
    return 'background-color: %s; color: black; border: 1px solid black' % color

def highlight_status(val):
    if val == 0:
        color = "red"
    elif val == 1:
        color = "blue"
    elif val == 2:
        color = "green"
    else:
        color = "grey"
    return 'background-color: %s; color: black; border: 1px solid black' % color    
df = pd.read_csv("results.csv")
model_profile = pd.read_csv("model_profile.csv").set_index("model")

In [38]:
ms = df.model.unique()
ns = df.namespace.unique()

mns = [m + "_" + n for m in ms for n in ns]


In [39]:


for scheduler in ("cosine", "const-cooldown"):
    ds, metric = "imagenet1k", "acc1"
    d = df
    d = d[d.pretrain_dataset=="datacomp_1b"]
    d = d[d.downstream_dataset==ds]
    d = d[d.epoch==d.total_epochs]

    d = d[d.lr_scheduler==scheduler]

    vars=("samples_seen_scale_pretty", "model_simple_namespace")

    d  = d.groupby(list(vars)).count().reset_index()
    #d = d.drop_duplicates(subset=vars, keep="first")
    d = pd.pivot(
        d,
        index="samples_seen_scale_pretty",
        columns=("model_simple_namespace"),
        values="path",
    )
    cols = d.columns
    cols = sorted(cols, key=lambda k:model_profile.loc["_".join(k.split("_")[:-1])].gflops)
    d = d[cols]
    for mn in set(mns) - set(cols):
        d[mn] = 0
    index = sorted(d.index, key=lambda k:samples_seen_from_str(k))
    d = d.loc[index]
    d = d.T
    d[d.isna()] = 0
    d = d.astype(int)
    d =  d.style.map(highlight_counts)
    print("Number finished experiments (trained and evaled) for scheduler", scheduler)
    display(d)

Number finished experiments (trained and evaled) for scheduler cosine


samples_seen_scale_pretty,1.28M,3M,6.4M,12.8M,30M,64M,128M,300M,640M,1.28B,3B,12.8B
model_simple_namespace,,,,,,,,,,,,
ViT-S-32_cap,10,0,0,8,0,0,6,0,0,12,0,0
ViT-S-32_clip,10,6,0,12,6,0,9,6,0,18,1,0
ViT-S-32_coca,3,2,0,3,4,0,4,4,0,0,0,0
ViT-S-32_mammut,12,15,19,10,17,31,8,14,22,11,1,0
ViT-M-32_cap,10,0,0,8,0,0,6,0,0,12,0,0
ViT-M-32_clip,10,6,0,12,6,0,9,6,0,18,1,0
ViT-M-32_coca,2,2,0,2,3,0,3,3,0,0,0,0
ViT-M-32_mammut,13,15,21,11,15,37,8,15,22,10,1,0
ViT-S-16_cap,10,0,0,8,0,0,6,0,0,12,0,0


Number finished experiments (trained and evaled) for scheduler const-cooldown


samples_seen_scale_pretty,1.28M,3M,6.4M,12.8M,64M,128M,640M,1.28B,3B,12.8B
model_simple_namespace,,,,,,,,,,
ViT-S-32-alt_clip,0,0,0,0,0,2,0,2,2,2
ViT-S-32_clip,64,64,64,63,6,9,24,3,3,3
ViT-M-32-alt_clip,0,0,0,0,0,2,0,2,2,2
ViT-M-32_clip,62,62,62,62,6,10,24,4,4,4
ViT-S-16_clip,0,0,0,0,0,4,0,4,4,4
ViT-B-32_clip,62,62,62,62,6,10,24,4,6,4
ViT-M-16-alt_clip,0,0,0,0,0,2,0,2,2,2
ViT-M-16_clip,0,0,0,0,0,2,0,2,2,2
ViT-B-16_clip,62,62,62,62,6,8,18,2,2,2
